<a href="https://colab.research.google.com/github/aaubs/ds-master/blob/main/notebooks/M5-db-sql-intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sqlite3
import pandas as pd

# Introduction to the Structured Quary LAnguage (SQL)

Text......

In [ ]:
!wget -O flights.db https://github.com/aaubs/ds-master/blob/main/data/flights.db?raw=true.1

In [ ]:
# Establish connection to DB
con = sqlite3.connect("flights.db")

#. Set up a cursor
cur = con.cursor()

In [ ]:
# Defining a query
sql_query = """
  SELECT name FROM sqlite_master
  WHERE type='table';
  """
    
# executing our sql query
cur.execute(sql_query)

In [ ]:
# Fetching teh results
cur.fetchall()

We have three different tables in the database.



In [ ]:
query = pd.read_sql_query('''select * from airports limit 5;''', con)
query

In [ ]:
query = pd.read_sql_query('''select * from airlines limit 5;''', con)
query

In [ ]:
query = pd.read_sql_query('''select * from routes limit 5;''', con)
query